In [139]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
from qiskit_metal import draw, Dict,designs
from qiskit_metal.qlibrary.core import BaseQubit
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.draw import LineString
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.core.qroute import QRouteLead, QRoutePoint, QRoute
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.resonator.readoutres_fc import ReadoutResFC
from  qiskit_metal.qlibrary.user_components.my_qcomponent import  New_Transomon_Cross, RouteConnector,MyReadoutRes01,MyFluxLine01,MyFluxLine02,MyConnector,MyXYLine01,MyCircle
from  qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
import  math
from collections import  OrderedDict
import numpy as np
import time

In [141]:
# Initialise design
design = designs.DesignPlanar()
# Specify design name
design.metadata['design_name'] = 'FlipChip_Device'
# launch GUI
gui = MetalGUI(design)
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

Logger issue: Internal C++ object (QTextEditLogger) already deleted.


In [142]:
design.chips['main']['material'] = 'sapphire'
design.chips['main']['size']['size_x'] = '30 mm'
design.chips['main']['size']['size_y'] = '30mm'
design.variables.cpw_gap='5 um'
# design.chips,design.variables
my_chip = MyCircle(design,'my_chip', options=Dict(radius='35mm'))

In [143]:
# design the layout of launchpad
points = []
N=42
size = 32.0
pad_pad_space = 0.7
edge_gap = (size -(pad_pad_space*(N-1)))/2
for i in range(N):
    shape = draw.Point(-size/2+edge_gap+i*pad_pad_space,size/2)
    points.append(shape)
# points.append()
# for i in range(N):
#     shape = draw.Point(size/2,size/2-i*size/N)
x = draw.shapely.geometrycollections(points)
x0 = draw.rotate(x,90,origin=(0,0))
x1 = draw.rotate(x0,90,origin=(0,0))
x2 = draw.rotate(x1,90,origin=(0,0))
square = draw.shapely.geometrycollections([x,x0,x1,x2])
square = draw.rotate(square,45,origin=(0,0))
square_coords =[]
for i in range(4):
    for j in range(N):
        square_coords.append(square.geoms[i].geoms[j].coords[0])


In [144]:
opt=Dict(pos_x=0 , pos_y=0, orientation='-45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_a=Dict(pos_x=0 , pos_y=0, orientation='45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_b=Dict(pos_x=0 , pos_y=0, orientation='135', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_c=Dict(pos_x=0 , pos_y=0, orientation='-135', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
# test = OpenToGround(design, 'open01', options=Dict(pos_x='-3 mm',  pos_y=pos_y_zline+0.02, orientation='-45', chip ='C_chip'),)
launch_zline = LaunchpadWirebond(design,'launch_zline',options=opt)
launch_zline_a = LaunchpadWirebond(design,'launch_zline_a',options=opt_a)
launch_zline_b = LaunchpadWirebond(design,'launch_zline_b',options=opt_b)
launch_zline_c = LaunchpadWirebond(design,'launch_zline_c',options=opt_c)
# gui.rebuild()
# gui.autoscale()

In [145]:
design.delete_all_components()
launch_list = []
for i in range(4):
    for j in range(N):
         if (i==0):
            launch_list.append(design.copy_qcomponent(launch_zline,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         elif (i==1):
             launch_list.append(design.copy_qcomponent(launch_zline_a,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         elif (i==2):
             launch_list.append(design.copy_qcomponent(launch_zline_b,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         else:
             launch_list.append(design.copy_qcomponent(launch_zline_c,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
# launch_list.append(design.copy_qcomponent(launch_zline,'launch_zline'+str(15), Dict(pos_x =square_coords[15][0] ,pos_y=square_coords[15][1])))
gui.rebuild()
gui.autoscale()

Logger issue: Internal C++ object (QTextEditLogger) already deleted.


In [146]:
# draw an Xmon on the Q_chip. Notice that I have defined one more item called chip and set it to be the 'Q_chip'
# position the center of the Xmon at (0,0)
q0_x = 0
q0_y = 0

# build the device, positioned at (q1_x, q1_y)
# q1 = TransmonCross(design, 'Q1', options = Dict(pos_x=q1_x, pos_y=q1_y, **options))
q0 = New_Transomon_Cross(design, 'Q0', options = Dict(pos_x=q0_x, pos_y=q0_y, layer='1'))
q0.options.gds_cell_name = 'FakeJunction_01'

# rebuild in gui
gui.rebuild()
gui.autoscale()

In [147]:
#design the 100 qubits layout
qq_space = 0.015
qubit_num = 18
total_qubit_num = 101
qubit_num0= qubit_num+5
qubit_pos_list = []
y1 = q0.parse_options().cross_width+qq_space
y2 = -1*(q0.parse_options().cross_width+qq_space)
x1 = -int((qubit_num)/4)*y1          #divide qubits into four parts for location calculation

q0.options.pos_y = -y1
q0_y =q0.options.pos_y


for i in range(int(qubit_num)):
    if  (i<(qubit_num)/2):
        qubit_pos_list.append((q0_x+x1+i*y1,q0_y+y1))
    else:
        qubit_pos_list.append((q0_x+x1+(i-qubit_num/2)*y1,q0_y+y2))

q0.options.pos_y = -y1
q0_y =q0.options.pos_y

# qubit_pos_list

In [148]:

qubit_list = []
qubit_list.append(q0)
for i in range(int(qubit_num)):
     qubit_list.append(design.copy_qcomponent(q0,'Q'+str(i+1), Dict(pos_x =qubit_pos_list[i][0] ,pos_y=qubit_pos_list[i][1])))

qubit_list.append(design.copy_qcomponent(q0,'Q19', Dict(pos_x =q0_x+x1 ,pos_y=q0_y)))
qubit_list.append(design.copy_qcomponent(q0,'Q20', Dict(pos_x =-(q0_x+x1) ,pos_y=q0_y)))
qubit_list.append(design.copy_qcomponent(q0,'Q21', Dict(pos_x =q0_x+x1/2 ,pos_y=-2*y1+q0_y)))
qubit_list.append(design.copy_qcomponent(q0,'Q22', Dict(pos_x =-x1/2+q0_x ,pos_y=-2*y1+q0_y)))

#draw first part of 100 qubits
qubit_name = []
for i in range(int(qubit_num0)):
    qubit_name.append('Q'+str(i+qubit_num0))
qubit_pos_dict_list =[]
for i in range(int(qubit_num0)):
    qubit_pos_dict_list.append(dict(pos_y = design.components['Q'+str(i)].parse_options().pos_y+4*y1))
qubits_copy1 = design.copy_multiple_qcomponents(qubit_list,qubit_name,qubit_pos_dict_list)

#draw second part
for i in range(int(qubit_num0)):
    qubit_name.append('Q'+str(i+2*qubit_num0))
# qubit_pos_dict_list =[]
for i in range(int(qubit_num0)):
    qubit_pos_dict_list.append(dict(pos_y = design.components['Q'+str(i)].parse_options().pos_y+8*y1))
qubits_copy1 = design.copy_multiple_qcomponents(qubit_list,qubit_name[23:],qubit_pos_dict_list[23:])

#draw third part
for i in range(int(qubit_num0)):
    qubit_name.append('Q'+str(i+3*qubit_num0))
# qubit_pos_dict_list =[]
for i in range(int(qubit_num0)):
    qubit_pos_dict_list.append(dict(pos_y = design.components['Q'+str(i)].parse_options().pos_y-4*y1))
qubits_copy1 = design.copy_multiple_qcomponents(qubit_list,qubit_name[46:],qubit_pos_dict_list[46:])

#draw last part
for i in range(int(qubit_num/2)):
    qubit_name.append('Q'+str(i+4*qubit_num0))
# qubit_pos_dict_list =[]
for i in range(int(qubit_num/2)):
    qubit_pos_dict_list.append(dict(pos_y = design.components['Q'+str(i+1)].parse_options().pos_y-8*y1))
qubits_copy1 = design.copy_multiple_qcomponents(qubit_list[1:int(qubit_num/2+1)],qubit_name[69:],qubit_pos_dict_list[69:])
my_chip = MyCircle(design,'my_chip', options=Dict(radius='35mm'))
gui.rebuild()
gui.autoscale()

In [149]:
# add the readout resonators
options = Dict(
       readout_radius='50 um',
       readout_cpw_width='10 um',
       readout_cpw_gap='5 um',
       readout_cpw_turnradius='27 um',
       vertical_start_length = '40 um',
       vertical_end_length = '300 um',
       horizontal_start_length01= '400 um',
       horizontal_start_length02 = '400 um',
       horizontal_end_length = '500 um',
       total_length = '4200 um',
       arc_step='1 um',
       meander_round = '5',
       orientation='0',
       layer='2',
       layer_subtract='2',
       inverse = False,
       mirror = False,
       subtract=True,
       chip='main',)
location_x = design.components['Q0'].parse_options().cross_width/4
# the resonator is set to have its origin at the center of the circular patch.
# So we set the qubit and the resonator to share the same coordinate (q1_x, q1_y)
r0 = MyReadoutRes01(design, 'R0', options = Dict(pos_x = design.components['Q0'].parse_options().pos_x+location_x, pos_y = design.components['Q0'].parse_options().pos_y, **options))

resonator_list = []
resonator_list.append(r0)
for i in range(int(total_qubit_num)-1):
     location = design.components['Q'+str(i+1)] .parse_options().cross_width/4
     resonator_list.append(design.copy_qcomponent(r0,'R'+str(i+1), Dict(pos_x =design.components['Q'+str(i+1)].parse_options().pos_x+location,pos_y=design.components['Q'+str(i+1)].parse_options().pos_y)))

gui.rebuild()
gui.autoscale()

In [ ]:
rr_space = 0.025
design.components['R0'].options.mirror = True
design.components['R0'].options.inverse = True
design.components['R0'].options.meander_round = '3'

r_0 = design.components['R0'].parse_options().readout_radius
r = design.components['R0'].parse_options().readout_cpw_turnradius
l_2 = design.components['R0'].parse_options().vertical_start_length
l_6 = design.components['R0'].parse_options().vertical_end_length
turn_round_n = design.components['R0'].parse_options().meander_round
# vertical_length = r_0+ l_2+2*r*(turn_round_n+1)+l_6

l_v = design.components['R5'].pins.readout.middle[1]-2*rr_space-design.components['R0'].parse_options().pos_y-r_0-l_2-2*r*(turn_round_n+2.5)
design.components['R0'].options.vertical_end_length = l_v

flip_resonator_list = np.array([0,19,20,21,22])
for i in range(3):
    flip_resonator_list = np.concatenate([flip_resonator_list,flip_resonator_list+23])
for i in range(9):
    flip_resonator_list = np.append(flip_resonator_list,92+i)
for i in flip_resonator_list:
    design.components['R'+str(i)].options.mirror = True
    design.components['R'+str(i)].options.inverse = True
    design.components['R'+str(i)].options.meander_round = '3'
    design.components['R'+str(i)].options.vertical_end_length = l_v

gui.rebuild()
gui.autoscale()

In [ ]:
# design.components['Q0'].parse_options()

In [ ]:
%%time
#set intermediate pins (close to qubits) for layout convenience
unit_y = design.components['Q0'].parse_options().cross_width+qq_space
pin_qubit_list=[]
pin_qubit_num0 = 9
pin_edge_space = 0.3   #horizontal left space for ease
pin_start_space = 0.15   #vertical left space for ease
pin_end_space = 0.25
pos_start_x = design.components['Q65'].parse_options().pos_x-design.components['Q65'].parse_options().cross_width/2-pin_edge_space
pos_start_x_r = design.components['Q66'].parse_options().pos_x+design.components['Q66'].parse_options().cross_width/2+pin_edge_space
# pos_start_y = design.components['Q65'].parse_options().pos_y-pin_start_space
pos_end_x = pos_start_x
# pos_end_y = design.components['Q56'].parse_options().pos_y+pin_end_space
# pin_pin_length = abs(pos_start_y-pos_end_y)
# pin_pin_space = pin_pin_length/(pin_qubit_num0-1)

def set_side_pins(pos_start_x, pin_qubit_num, top_qubit_id, low_qubit_id):
    pos_start_y = design.components['Q'+str(top_qubit_id)].parse_options().pos_y-pin_start_space
    pos_end_y = design.components['Q'+str(low_qubit_id)].parse_options().pos_y+pin_end_space
    pin_pin_space = abs(pos_start_y-pos_end_y)/(pin_qubit_num-1)
    for i in range(pin_qubit_num):
        otg = OpenToGround(design, 'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_'+str(i),
                options=Dict(pos_x=pos_start_x,  pos_y=pos_start_y-i*pin_pin_space, orientation='0'))
        otg1 = OpenToGround(design, 'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_rhs'+str(i),
        options=Dict(pos_x=pos_start_x,  pos_y=pos_start_y-i*pin_pin_space, orientation='180'))
        pin_qubit_list.append(otg)
        pin_qubit_list.append(otg1)
    return pos_start_y,pos_end_y,pin_pin_space

def set_enlarged_side_pins(pos_start_y, pin_qubit_num,pin_pin_space,pin_pin_space_l,pos_start_x_l, top_qubit_id,low_qubit_id):
    pos_start_y_l = pos_start_y-int((pin_qubit_num)/2)*pin_pin_space+int((pin_qubit_num)/2)*pin_pin_space_l
    # pos_end_y_l = pos_end_y-(pin_qubit_num-1)*pin_pin_space_l
    for i in range(pin_qubit_num):
        otg = OpenToGround(design, 'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r'+str(i),
                                            options=Dict(pos_x=pos_start_x_l,  pos_y=pos_start_y_l-i*pin_pin_space_l, orientation='180'))
        pin_qubit_list.append(otg)
    for i in range(pin_qubit_num):
        otg = OpenToGround(design, 'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_l'+str(i),
                           options=Dict(pos_x=pos_start_x_l,  pos_y=pos_start_y_l-i*pin_pin_space_l, orientation='0'))
        pin_qubit_list.append(otg)


def  set_enlarged_readout_line_pins(pos_start_x_l, pin_pin_space_l, top_qubit_id,low_qubit_id,readout_id): # for qubit_num=8
    pin_qubit_list.append(OpenToGround(design, 'open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(readout_id)+'_r', options=Dict(pos_x=pos_start_x_l,  pos_y=design.components['open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r0'].parse_options().pos_y+pin_pin_space_l, orientation='180')))
    pin_qubit_list.append(OpenToGround(design,  'open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(readout_id)+'_l', options=Dict(pos_x=pos_start_x_l,  pos_y=design.components['open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r0'].parse_options().pos_y+pin_pin_space_l, orientation='0')))



pin_qubit_num0_r = 8
#1st section pins with sides and top wiring
set_side_pins(pos_start_x, pin_qubit_num0,65,56)
set_side_pins(pos_start_x_r,pin_qubit_num0_r,66,64)

#2nd section side pins
pin_qubit_num1 = 8
pin_qubit_num1_r = 8
pos_start_y,pos_end_y,pin_pin_space = set_side_pins(pos_start_x,pin_qubit_num1,67,24)

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num1_r,68,32)

# enlarge pins
enlarge_xspace = 1.5
enlarge_ratio = 2
pin_pin_length = abs(pos_start_y-pos_end_y)
pin_pin_length_l = pin_pin_length*enlarge_ratio
pin_pin_space_l = pin_pin_length_l/(pin_qubit_num1-1)
pos_start_x_l = pos_start_x-enlarge_xspace
set_enlarged_side_pins(pos_start_y, pin_qubit_num1,pin_pin_space,pin_pin_space_l,pos_start_x_l,67,24)
set_enlarged_readout_line_pins(pos_start_x_l,pin_pin_space_l,67,24,0)

pin_pin_space_l_r = pin_pin_length_l/(pin_qubit_num1_r-1)
pos_start_x_l_r = pos_start_x_r+enlarge_xspace
set_enlarged_side_pins(pos_start_y_r, pin_qubit_num1_r,pin_pin_space_r,pin_pin_space_l_r,pos_start_x_l_r,68,32)
set_enlarged_readout_line_pins(pos_start_x_l_r,pin_pin_space_l_r, 68,32,0)


#3rd section side pins
pin_qubit_num2 = 8
pin_qubit_num2_r = 9
pos_start_y,pos_end_y,pin_pin_space=set_side_pins(pos_start_x,pin_qubit_num2,42,33)

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num2_r,43,41)

#enlarge pins
pin_pin_space_l = pin_pin_length_l/(pin_qubit_num2-1)
set_enlarged_side_pins(pos_start_y,pin_qubit_num2,pin_pin_space,pin_pin_space_l,pos_start_x_l,42,33)
set_enlarged_readout_line_pins(pos_start_x_l,pin_pin_space_l,42,33,1)

pin_pin_space_l_r = pin_pin_length_l/(pin_qubit_num2_r-1)
set_enlarged_side_pins(pos_start_y_r, pin_qubit_num2_r,pin_pin_space_r,pin_pin_space_l_r,pos_start_x_l_r,43,41)
set_enlarged_readout_line_pins(pos_start_x_l_r,pin_pin_space_l_r, 43,41,1)

#fourth section side pins
pin_qubit_num3 = 8
pin_qubit_num3_r = 8
pos_start_y,pos_end_y,pin_pin_space=set_side_pins(pos_start_x,pin_qubit_num3,44,1)

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num3_r,45,9)

#enlarge pins
pin_pin_space_l = pin_pin_length_l/(pin_qubit_num3-1)
set_enlarged_side_pins(pos_start_y,pin_qubit_num3,pin_pin_space,pin_pin_space_l,pos_start_x_l,44,1)
set_enlarged_readout_line_pins(pos_start_x_l,pin_pin_space_l,44,1,2)

pin_pin_space_l_r = pin_pin_length_l/(pin_qubit_num3_r-1)
set_enlarged_side_pins(pos_start_y_r, pin_qubit_num3_r,pin_pin_space_r,pin_pin_space_l_r,pos_start_x_l_r,45,9)
set_enlarged_readout_line_pins(pos_start_x_l_r,pin_pin_space_l_r, 45,9,2)


# 5th section side pins
pin_qubit_num4 = 9
pin_qubit_num4_r = 8
pos_start_y,pos_end_y,pin_pin_space = set_side_pins(pos_start_x,pin_qubit_num4,19,10)

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num4_r,20,18)

#enlarge pins
pin_pin_space_l = pin_pin_length_l/(pin_qubit_num4-1)
set_enlarged_side_pins(pos_start_y,pin_qubit_num4,pin_pin_space,pin_pin_space_l,pos_start_x_l,19,10)
set_enlarged_readout_line_pins(pos_start_x_l,pin_pin_space_l,19,10,3)

pin_pin_space_l_r = pin_pin_length_l/(pin_qubit_num4_r-1)
set_enlarged_side_pins(pos_start_y_r, pin_qubit_num4_r,pin_pin_space_r,pin_pin_space_l_r,pos_start_x_l_r,20,18)
set_enlarged_readout_line_pins(pos_start_x_l_r,pin_pin_space_l_r, 20,18,3)

#6th section side pins
pin_qubit_num5 = 8
pin_qubit_num5_r =8
pos_start_y,pos_end_y,pin_pin_space = set_side_pins(pos_start_x,pin_qubit_num5,21,70)

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num5_r,22,78)


#enlarge pins
pin_pin_space_l = pin_pin_length_l/(pin_qubit_num5-1)
set_enlarged_side_pins(pos_start_y,pin_qubit_num5,pin_pin_space,pin_pin_space_l,pos_start_x_l,21,70)
set_enlarged_readout_line_pins(pos_start_x_l,pin_pin_space_l, 21,70,4)

pin_pin_space_l_r = pin_pin_length_l/(pin_qubit_num5_r-1)
set_enlarged_side_pins(pos_start_y_r, pin_qubit_num5_r,pin_pin_space_r,pin_pin_space_l_r,pos_start_x_l_r,22,78)
set_enlarged_readout_line_pins(pos_start_x_l_r,pin_pin_space_l_r, 22,78,4)

#7th section side pins
pin_qubit_num6 = 8
pin_qubit_num6_r = 9
pos_start_y,pos_end_y,pin_pin_space = set_side_pins(pos_start_x,pin_qubit_num6,88,79)

pin_qubit_list.append(OpenToGround(design, 'open_Q90', options=Dict(pos_x=pos_start_x,  pos_y=design.components['Q90'].parse_options().pos_y-pin_start_space, orientation='0')))
pin_qubit_list.append(OpenToGround(design, 'open_Q90_r', options=Dict(pos_x=pos_start_x,  pos_y=design.components['Q90'].parse_options().pos_y-pin_start_space, orientation='180')))

pos_start_y_r,pos_end_y_r,pin_pin_space_r = set_side_pins(pos_start_x_r,pin_qubit_num6_r,89,87)

pin_qubit_list.append(OpenToGround(design, 'open_Q91', options=Dict(pos_x=pos_start_x_r,  pos_y=design.components['Q91'].parse_options().pos_y-pin_start_space, orientation='0')))
pin_qubit_list.append(OpenToGround(design, 'open_Q91_r', options=Dict(pos_x=pos_start_x_r,  pos_y=design.components['Q91'].parse_options().pos_y-pin_start_space, orientation='180')))


#set the read line pins
readline_pos_y_list =[]
for i in [0,23,46,69]:
    readline_pos_y_list.append(design.components['R'+str(i)].pins.readout.middle[1]+(design.components['R'+str(i+5)].pins.readout.middle[1]-design.components['R'+str(i)].pins.readout.middle[1])/2)
for i in [21,44,67,90]:
    readline_pos_y_list.append(design.components['R'+str(i)].pins.readout.middle[1]+(design.components['R'+str(i-9)].pins.readout.middle[1]-design.components['R'+str(i)].pins.readout.middle[1])/2)
readline_pos_y_list.append(design.components['R92'].pins.readout.middle[1]+rr_space)

for i in range(len(readline_pos_y_list)):
    otg0 = OpenToGround(design, 'open_readout_line_l'+str(i), options=Dict(pos_x=pos_start_x,  pos_y=readline_pos_y_list[i], orientation='0'))
    otg0_r = OpenToGround(design, 'open_readout_line_l_rpin'+str(i), options=Dict(pos_x=pos_start_x,  pos_y=readline_pos_y_list[i], orientation='180'))
    otg1 = OpenToGround(design, 'open_readout_line_r'+str(i), options=Dict(pos_x=-pos_start_x,  pos_y=readline_pos_y_list[i], orientation='0'))
    otg1_r = OpenToGround(design, 'open_readout_line_r_rpin'+str(i), options=Dict(pos_x=-pos_start_x,  pos_y=readline_pos_y_list[i], orientation='180'))
    pin_qubit_list.append(otg0)
    pin_qubit_list.append(otg1)

# set the top pins for virtual wiring
top_vpin_list = []
top_vpin_launch_list = []
pad_pin_vspace =  1.5
top_pin_num =7
pin_for_side_num = 3
top_vpin_num = pin_qubit_num0+top_pin_num-pin_for_side_num+1

top_vpin_pos_y = design.components['launch_zline029'].parse_options().pos_y-pad_pin_vspace
top_vpin_start_pos_x = design.components['Q47'].parse_options().pos_x
top_vpin_end_pos_x = design.components['Q51'].parse_options().pos_x-0.1
top_vpin_space =abs( top_vpin_end_pos_x-top_vpin_start_pos_x)/(top_vpin_num-1)

for i in range(top_vpin_num):
    stg = OpenToGround(design, 'top_virtual'+str(i), options=Dict(pos_x=top_vpin_start_pos_x+i*top_vpin_space,  pos_y=top_vpin_pos_y, orientation='90'))
    stg1 = OpenToGround(design, 'top_virtual_r'+str(i), options=Dict(pos_x=-(top_vpin_start_pos_x+i*top_vpin_space),  pos_y=top_vpin_pos_y, orientation='90'))
    top_vpin_list.append(stg)
    top_vpin_list.append(stg1)
    stg2 = OpenToGround(design, 'top_virtual_l'+str(i), options=Dict(pos_x=top_vpin_start_pos_x+i*top_vpin_space,  pos_y=top_vpin_pos_y, orientation='-90'))
    stg3 = OpenToGround(design, 'top_virtual_l_r'+str(i), options=Dict(pos_x=-(top_vpin_start_pos_x+i*top_vpin_space),  pos_y=top_vpin_pos_y, orientation='-90'))
    top_vpin_launch_list.append(stg2)
    top_vpin_launch_list.append(stg3)


# for i in range(top_vpin_num):
#     stg = OpenToGround(design, 'top_virtual_l'+str(i), options=Dict(pos_x=top_vpin_start_pos_x+i*top_vpin_space,  pos_y=top_vpin_pos_y, orientation='-90'))
#     top_vpin_launch_list.append(stg)
low_vpin_num = top_vpin_num+2
low_vpin_pos_y = design.components['launch_zline113'].parse_options().pos_y+pad_pin_vspace-0.5
low_vpin_start_pos_x = design.components['Q92'].parse_options().pos_x
low_vpin_end_pos_x = design.components['Q96'].parse_options().pos_x-0.1
low_vpin_space =abs( low_vpin_end_pos_x-low_vpin_start_pos_x)/(low_vpin_num-1)
for i in range(low_vpin_num):
    stg = OpenToGround(design, 'low_virtual'+str(i), options=Dict(pos_x=low_vpin_start_pos_x+i*low_vpin_space,  pos_y=low_vpin_pos_y, orientation='-90'))
    stg1 = OpenToGround(design, 'low_virtual_r'+str(i), options=Dict(pos_x=-(low_vpin_start_pos_x+i*low_vpin_space),  pos_y=low_vpin_pos_y, orientation='-90'))
    top_vpin_list.append(stg)
    top_vpin_list.append(stg1)
    stg2 = OpenToGround(design, 'low_virtual_l'+str(i), options=Dict(pos_x=low_vpin_start_pos_x+i*low_vpin_space,  pos_y=low_vpin_pos_y, orientation='90'))
    stg3 = OpenToGround(design, 'low_virtual_l_r'+str(i), options=Dict(pos_x=-(low_vpin_start_pos_x+i*low_vpin_space),  pos_y=low_vpin_pos_y, orientation='90'))
    top_vpin_launch_list.append(stg2)
    top_vpin_launch_list.append(stg3)

gui.rebuild()
gui.autoscale()

# design.components['Q2'].parse_options().pos_y

In [ ]:
# set the enlarged pins


In [ ]:
# len(readline_pos_y_list),readline_pos_y_list,design.components['R46'].pins.readout.middle[1]+(design.components['R51'].pins.readout.middle[1]-design.components['R0'].pins.readout.middle[1])/2# pos_start_y_l, design.components['Q67'].parse_options().pos_y-pin_start_space,pin_pin_space

In [ ]:
#wiring first 16 pins (both side and top)
def  get_extended_pos_y(y1,length,ydirction):  #actually it can also get x position, will correct name in future
    y_extended = y1+length*ydirction
    return y_extended
fillet = '90 um'
extended_pin_length = 0.09

extended_pin_length_end = 0.2
x_launch_zline28 =get_extended_pos_y(design.components['launch_zline028'].pins.tie.middle[0],extended_pin_length,
                                              design.components['launch_zline028'].pins.tie.normal[0])
x_launch_zline25 =get_extended_pos_y(design.components['launch_zline025'].pins.tie.middle[0],extended_pin_length,
                                              design.components['launch_zline025'].pins.tie.normal[0])
side_pin_num0 = pin_qubit_num0-pin_for_side_num
left_eps = 0.15  #length left for no collision at sides

# wiring_space0 =( abs(x_launch_zline28-design.components['open_Q65_Q56_5'].parse_options().pos_x)-left_eps)/side_pin_num0
# extended_pin_length_start =wiring_space0*(side_pin_num0-1)+left_eps
total_wiring_space =abs(x_launch_zline25-design.components['open_Q65_Q56_8'].parse_options().pos_x)
wiring_space0 =( total_wiring_space-left_eps)/(pin_qubit_num0 )
extended_pin_length_start =wiring_space0*(side_pin_num0-1)+left_eps


side_pin_list=[]
# for i in range(3):
#     # y_launch_zline26 =get_extended_pos_y(design.components['launch_zline26'].pins.tie.middle[1],extended_pin_length,
#     #                                               design.components['launch_zline26'].pins.tie.normal[1])
#     jogsS = OrderedDict()
#     jogsS[0] = ["R", str(8-i)+"mm"]
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_'+str(8-i),pin='open'),
#                              end_pin=Dict(component='launch_zline0'+str(26+i),pin='tie'),),lead = Dict(start_straight=total_wiring_space-i*wiring_space0,
#                              end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
#     side_pin_list.append(RoutePathfinder(design,'line_launch'+str(26+i)+'_pin0'+str(8-i),options=pin_opt))
jogsS = OrderedDict()
jogsS[0] = ["R", '100um']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_8',pin='open'),
                         end_pin=Dict(component='launch_zline025',pin='tie'),),lead = Dict(start_straight=total_wiring_space,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch25'+'_pin08',options=pin_opt))

jogsS = OrderedDict()
jogsS[0] = ["R", '4.6mm']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_7',pin='open'),
                         end_pin=Dict(component='launch_zline026',pin='tie'),),lead = Dict(start_straight=total_wiring_space-wiring_space0,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch26'+'_pin07',options=pin_opt))


jogsS[0] = ["R", '4.3mm']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_6',pin='open'),
                         end_pin=Dict(component='launch_zline027',pin='tie'),),lead = Dict(start_straight=total_wiring_space-2*wiring_space0,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch27'+'_pin06',options=pin_opt))

gui.rebuild()
gui.autoscale()

In [ ]:

# wiring_space0,total_wiring_space

In [ ]:
# wiring for top pin


# extended_pin_length_start


# side_top_pin_list = []
# for i in [5,0,1,2,3,4,]:
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_'+str(i),pin='open'),
#                          end_pin=Dict(component='top_virtual'+str(5-i),pin='open'),),lead = Dict(start_straight=left_eps+(i+1)*wiring_space0,
#                         end_straight = (side_pin_num0-i)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
#     side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(i)+'_top_vpin'+str(5-i),options=pin_opt))
# gui.rebuild()
# gui.autoscale()


In [ ]:
%%time
#wiring from side to top

# for i in range(side_pin_num0):
#     design.delete_component('line_pin'+str(5-i)+'_top_vpin'+str(i))
# design.delete_component('readout_line_l2_top_vpin6')
# total_wiring_space =abs(x_launch_zline25-design.components['open_Q65_Q56_8'].parse_options().pos_x)
# wiring_space0 =( total_wiring_space-left_eps)/(pin_qubit_num0 )
# extended_pin_length_start =wiring_space0*(side_pin_num0-1)+left_eps

side_top_pin_list = []
jogsS = OrderedDict()
jogsS[0] = ["R", '4.0mm']
jogsS[1] = ["R", 2*wiring_space0]
jogsS[2] = ["L", '100um']
jogsE = OrderedDict()
jogsE[0] = ["R", '100um']
for i in range(side_pin_num0):
    jogsS[0] = ["R", str(4-0.3*i)+'mm']
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_'+str(5-i),pin='open'),
                     end_pin=Dict(component='top_virtual'+str(i),pin='open'),),lead = Dict(start_straight=total_wiring_space-(3+i)*wiring_space0,
                     end_straight = (side_pin_num0-5+i)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
                     fillet=fillet, chip = 'main')
    side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(5-i)+'_top_vpin'+str(i),options=pin_opt))

readout_open_space = design.components['open_readout_line_l2'].parse_options().pos_y - design.components['open_Q65_Q56_0'].parse_options().pos_y
jogsS[0] = ["R", str(2.2-readout_open_space)+'mm']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l2',pin='open'),
                 end_pin=Dict(component='top_virtual6',pin='open'),),lead = Dict(start_straight=total_wiring_space-(pin_qubit_num0)*wiring_space0,
                 end_straight = (side_pin_num0+1)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
                 fillet=fillet, chip = 'main')
side_top_pin_list.append(RoutePathfinder(design,'readout_line_l2_top_vpin6',options=pin_opt))

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_5',pin='open'),
#                      end_pin=Dict(component='top_virtual0',pin='open'),),lead = Dict(start_straight=total_wiring_space-3*wiring_space0,
#                      end_straight = (side_pin_num0-5)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin5'+'_top_vpin0',options=pin_opt))
#
# jogsS[0] = ["R", '3.7mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_4',pin='open'),
#                      end_pin=Dict(component='top_virtual1',pin='open'),),lead = Dict(start_straight=total_wiring_space-4*wiring_space0,
#                      end_straight = (side_pin_num0-4)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin4'+'_top_vpin1',options=pin_opt))
#
# jogsS[0] = ["R", '3.4mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_3',pin='open'),
#                      end_pin=Dict(component='top_virtual2',pin='open'),),lead = Dict(start_straight=total_wiring_space-5*wiring_space0,
#                      end_straight = (side_pin_num0-3)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin3'+'_top_vpin2',options=pin_opt))
#
# jogsS[0] = ["R", '3.1mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_2',pin='open'),
#                      end_pin=Dict(component='top_virtual3',pin='open'),),lead = Dict(start_straight=total_wiring_space-6*wiring_space0,
#                      end_straight = (side_pin_num0-2)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin2'+'_top_vpin3',options=pin_opt))
#
# jogsS[0] = ["R", '2.8mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_1',pin='open'),
#                      end_pin=Dict(component='top_virtual4',pin='open'),),lead = Dict(start_straight=total_wiring_space-7*wiring_space0,
#                      end_straight = (side_pin_num0-1)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin1'+'_top_vpin4',options=pin_opt))
#
# jogsS[0] = ["R", '2.5mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_0',pin='open'),
#                      end_pin=Dict(component='top_virtual5',pin='open'),),lead = Dict(start_straight=total_wiring_space-8*wiring_space0,
#                      end_straight = (side_pin_num0)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                      fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'line_pin0'+'_top_vpin5',options=pin_opt))

gui.rebuild()
gui.autoscale()

In [ ]:
%%time
#wiring for side pin ----part 1: enlarge pin space
pin_side_list=[]
fillet_l = '50 um'
# end_straight_step = (enlarge_xspace-0.1)*2/pin_qubit_num1
end_straight_step = 0.2
start_straight = 0.1
jogsS = OrderedDict()

def  routing_enlarge_side_pin(pin_qubit_num,top_qubit_id,low_qubit_id):
    for i in range((pin_qubit_num)):
        if (i<int(pin_qubit_num/2)):
            pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_'+str(i),pin='open'),
                         end_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
                         end_straight = enlarge_xspace-2*start_straight-i*end_straight_step,), fillet=fillet_l, chip = 'main')
            pin_side_list.append(RoutePathfinder(design,'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(i)+'_pin'+str(i),options=pin_opt))
        elif (i>int(pin_qubit_num/2)):
            pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_'+str(i),pin='open'),
                         end_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
                         end_straight = enlarge_xspace-2*start_straight-(pin_qubit_num-i)*end_straight_step,), fillet=fillet_l, chip = 'main')
            pin_side_list.append(RoutePathfinder(design,'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(i)+'_pin'+str(i),options=pin_opt))
        else:
            pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_'+str(i),pin='open'),
                         end_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_r'+str(i),pin='open'),), fillet=fillet_l, chip = 'main')
            pin_side_list.append(RoutePathfinder(design,'open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(i)+'_pin'+str(i),options=pin_opt))

def  routing_readout_line_enlarged_pin(readout_line_id,enlarged_pin_id,top_qubit_id,low_qubit_id):
    jogsS[0] = ["R", '100um']
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l'+str(readout_line_id),pin='open'),
                 end_pin=Dict(component='open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(enlarged_pin_id)+'_r',pin='open'),), lead = Dict(start_straight=2*start_straight, end_straight = '100 um',start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
    pin_side_list.append(RoutePathfinder(design,'readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(readout_line_id)+'_r_pin'+str(enlarged_pin_id)+'_l',options=pin_opt))

 # 'open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(readout_id)+'_r'

routing_enlarge_side_pin(pin_qubit_num1,67,24)
routing_readout_line_enlarged_pin(6,0,67,24)
#
routing_enlarge_side_pin(pin_qubit_num2,42,33)
routing_readout_line_enlarged_pin(1,1,42,33)

routing_enlarge_side_pin(pin_qubit_num3,44,1)
routing_readout_line_enlarged_pin(5,2,44,1)

# for i in range((pin_qubit_num1)):
#     if (i<int(pin_qubit_num1/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-i*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     elif (i>int(pin_qubit_num1/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-(pin_qubit_num1-i)*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     else:
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r'+str(i),pin='open'),), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin'+str(i)+'_pin'+str(i),options=pin_opt))

# jogsS = OrderedDict()
# jogsS[0] = ["R", '100um']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l6',pin='open'),
#              end_pin=Dict(component='open_readout_line_pin0_r',pin='open'),), lead = Dict(start_straight=2*start_straight,
#             end_straight = '100 um',start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'readout_line6_r_pin0_l',options=pin_opt))


# for i in range(pin_qubit_num2):
#     if (i<int(pin_qubit_num2/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q42_Q33_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q42_Q33_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-i*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q42_Q33_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     elif (i>int(pin_qubit_num1/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q42_Q33_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q42_Q33_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-(pin_qubit_num2-i)*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q42_Q33_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     else:
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q42_Q33_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q42_Q33_r'+str(i),pin='open'),), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q42_Q33_pin'+str(i)+'_pin'+str(i),options=pin_opt))

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l1',pin='open'),
#              end_pin=Dict(component='open_readout_line_pin1_r',pin='open'),), lead = Dict(start_straight=2*start_straight,
#              end_straight = '100 um',start_jogged_extension=jogsS,),fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'readout_line1_r_pin1_l',options=pin_opt))


# for i in range(pin_qubit_num3):
#     if (i<int(pin_qubit_num3/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q44_Q1_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-i*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q44_Q1_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     elif (i>int(pin_qubit_num3/2)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q44_Q1_r'+str(i),pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-(pin_qubit_num3-i)*end_straight_step,), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q44_Q1_pin'+str(i)+'_pin'+str(i),options=pin_opt))
#     else:
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_'+str(i),pin='open'),
#                      end_pin=Dict(component='open_Q44_Q1_r'+str(i),pin='open'),), fillet=fillet_l, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'open_Q44_Q1_pin'+str(i)+'_pin'+str(i),options=pin_opt))

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l5',pin='open'),
#              end_pin=Dict(component='open_readout_line_pin2_r',pin='open'),),lead = Dict(start_straight=2*start_straight,
#              end_straight = '100 um', start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'readout_line5_r_pin2_l',options=pin_opt))

# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin0'+'_pin0',options=pin_opt))
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_0',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r0',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin0'+'_pin0',options=pin_opt))
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_1',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r1',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-end_straight_step,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin1'+'_pin1',options=pin_opt))
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_2',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r2',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-end_straight_step*2,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin2'+'_pin2',options=pin_opt))
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_3',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r3',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-end_straight_step*3,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin3'+'_pin3',options=pin_opt))
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_4',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r4',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-end_straight_step*4,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin4'+'_pin4',options=pin_opt))
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_5',pin='open'),
#                      end_pin=Dict(component='open_Q67_Q24_r5',pin='open'),),lead = Dict(start_straight=start_straight,
#                      end_straight = enlarge_xspace-2*start_straight-end_straight_step*3,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'open_Q67_Q24_pin5'+'_pin5',options=pin_opt))


gui.rebuild()
gui.autoscale()

In [ ]:
# routing_readout_line_enlarged_pin(6,0)

# jogsS = OrderedDict()
# jogsS[0] = ["R", '100um']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l6',pin='open'),
#              end_pin=Dict(component='open_readout_line_pin0_r',pin='open'),), lead = Dict(start_straight=2*start_straight,
#             end_straight = '100 um',start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'readout_line6_r_pin0_l',options=pin_opt))
#
#
# gui.rebuild()
# gui.autoscale()

In [161]:
%%time
#wiring for side pin ----part 1: enlarge pin space
def  routing_readout_line_enlarged_pin0(readout_line_id,enlarged_pin_id, top_qubit_id,low_qubit_id):
    jogsS[0] = ["R", '200um']
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l'+str(readout_line_id),pin='open'),
                 end_pin=Dict(component='open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(enlarged_pin_id)+'_r',pin='open'),), lead = Dict(start_straight=2*start_straight,
                end_straight = '100 um',start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
    pin_side_list.append(RoutePathfinder(design,'readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'+str(readout_line_id)+'_r_pin'+str(enlarged_pin_id)+'_l',options=pin_opt))


routing_enlarge_side_pin(pin_qubit_num4,19,10)
routing_readout_line_enlarged_pin0(0,3,19,10)

routing_enlarge_side_pin(pin_qubit_num5,21,70)
routing_readout_line_enlarged_pin(4,4,21,70)

# routing_enlarge_side_pin(pin_qubit_num6,88,79)
# routing_readout_line_enlarged_pin(3,5)
# fillet_l = '50 um'
# jogsS = OrderedDict()
# jogsS[0] = ["R", '100um']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l6',pin='open'),
#              end_pin=Dict(component='open_readout_line_pin0_r',pin='open'),), lead = Dict(start_straight=start_straight,
#             end_straight = '100 um',start_jogged_extension=jogsS,), fillet=fillet_l, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'readout_line6_r_pin0_l',options=pin_opt))
gui.rebuild()
gui.autoscale()

Wall time: 6min 18s


In [110]:
# design.components['open_Q67_Q24_l0'].pins

In [164]:
%%time
#wiring for side pin ---- part 2: wiring enlarged pins ----
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_l0',pin='open'),
#                         end_pin=Dict(component='launch_zline025',pin='tie'),),lead = Dict(start_straight=0.01,
#                         end_straight = extended_pin_length,),fillet=fillet, chip = 'main',)
# x = RoutePathfinder(design,'line_launch25'+'_pin0',options=pin_opt)
# design.delete_component('line_launch0_readout_line_pin0')
fillet = '90 um'
# extended_pin_length = 0.09
# pin_side_list = []

def routing_enlarged_pin_launchpad(pin_qubit_num, top_qubit_id,low_qubit_id,launchpad_id,launchpad_prefix,readout_line_id):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_pin'
                        +str(readout_line_id)+'_l',pin='open'), end_pin=Dict(component='launch_zline'+str(launchpad_prefix)+str(launchpad_id),pin='tie'),),lead = Dict(start_straight=0.01,  end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
    pin_side_list.append(RoutePathfinder(design,'line_launch'+str(launchpad_id)+'_readout_line_pin'+str(readout_line_id),options=pin_opt))

    for i in range(int(pin_qubit_num)):
        pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_l'+str(i),pin='open'),
                             end_pin=Dict(component='launch_zline'+str(launchpad_prefix)+str(int(launchpad_id-1-i)),pin='tie'),),lead = Dict(start_straight=0.01,
                             end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
        pin_side_list.append(RoutePathfinder(design,'line_launch'+str(int(launchpad_id-1-i))+'_pin'+str(i),options=pin_opt))


routing_enlarged_pin_launchpad(pin_qubit_num1,67,24,24,0,0)
routing_enlarged_pin_launchpad(pin_qubit_num2,42,33,15,0,1)
# routing_enlarged_pin_launchpad(pin_qubit_num1,67,24,0,24,0)

# #first section
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_pin0_l',pin='open'),
#                      end_pin=Dict(component='launch_zline0'+str(24),pin='tie'),),lead = Dict(start_straight=0.01,
#                      end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'line_launch24_readout_line_pin0',options=pin_opt))
#
# for i in range(int(pin_qubit_num1)):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q67_Q24_l'+str(i),pin='open'),
#                          end_pin=Dict(component='launch_zline0'+str(23-i),pin='tie'),),lead = Dict(start_straight=0.01,
#                          end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#     pin_side_list.append(RoutePathfinder(design,'line_launch'+str(23-i)+'_pin'+str(i),options=pin_opt))

# #second section
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_pin1_l',pin='open'),
#                      end_pin=Dict(component='launch_zline0'+str(15),pin='tie'),),lead = Dict(start_straight=0.01,
#                      end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
# pin_side_list.append(RoutePathfinder(design,'line_launch15_readout_line_pin1',options=pin_opt))
#
# for i in range(int(pin_qubit_num2)):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q42_Q33_l'+str(i),pin='open'),
#                          end_pin=Dict(component='launch_zline0'+str(14-i),pin='tie'),),lead = Dict(start_straight=0.01,
#                          end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#     pin_side_list.append(RoutePathfinder(design,'line_launch'+str(14-i)+'_pin'+str(i),options=pin_opt))

#third section
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_Q44_Q1_pin2_l',pin='open'),
                     end_pin=Dict(component='launch_zline0'+str(6),pin='tie'),),lead = Dict(start_straight=0.01,
                     end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
pin_side_list.append(RoutePathfinder(design,'line_launch6_readout_line_Q44_Q1_pin2',options=pin_opt))

for i in range(4):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_l'+str(i),pin='open'),
                         end_pin=Dict(component='launch_zline0'+str(5-i),pin='tie'),),lead = Dict(start_straight=0.01,
                         end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
    pin_side_list.append(RoutePathfinder(design,'line_launch'+str(5-i)+'_pin'+str(i),options=pin_opt))

# due to inadequate launchpad for one side, extend it to another side
# for i in range(4):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_l'+str(4+i),pin='open'),
#                          end_pin=Dict(component='launch_zline1'+str(41-i),pin='tie'),),lead = Dict(start_straight=0.01,
#                          end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#     pin_side_list.append(RoutePathfinder(design,'line_launch1'+str(41-i)+'_pin'+str(4+i),options=pin_opt))
for i in range(4):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_l'+str(4+i),pin='open'),
                         end_pin=Dict(component='launch_zline1'+str(41-i),pin='tie'),),lead = Dict(start_straight=0.01,
                         end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
    pin_side_list.append(RoutePathfinder(design,'line_launch1'+str(41-i)+'_pin'+str(4+i),options=pin_opt))
gui.rebuild()
gui.autoscale()

Wall time: 8min 30s


In [112]:

# for i in range(4):
#     design.delete_component('line_launch1'+str(41-i)+'_pin'+str(4+i))
#
# for i in range(4):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q44_Q1_l'+str(4+i),pin='open'),
#                          end_pin=Dict(component='launch_zline1'+str(39-i),pin='tie'),),lead = Dict(start_straight=0.01,
#                          end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#     pin_side_list.append(RoutePathfinder(design,'line_launch1'+str(39-i)+'_pin'+str(4+i),options=pin_opt))
#
# # pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_pin2_l',pin='open'),
# #                      end_pin=Dict(component='launch_zline0'+str(6),pin='tie'),),lead = Dict(start_straight=0.01,
# #                      end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
# # pin_side_list.append(RoutePathfinder(design,'line_launch6_readout_line_pin2',options=pin_opt))
# # # design.delete_component('line_launch139_pin7')
# gui.rebuild()
# gui.autoscale()

In [166]:
%%time
# design.delete_component('line_launch')
# def routing_enlarged_pin_launchpad(pin_qubit_num, top_qubit_id,low_qubit_id,launchpad_prefix,launchpad_id,readout_line_id):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_pin'+str(readout_line_id)+'_l',pin='open'),
#                          end_pin=Dict(component='launch_zline'+str(launchpad_prefix)+str(launchpad_id),pin='tie'),),lead = Dict(start_straight=0.01,
#                          end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#     pin_side_list.append(RoutePathfinder(design,'line_launch'+str(launchpad_id)+'_readout_line_pin'+str(readout_line_id),options=pin_opt))
#
#     for i in range(int(pin_qubit_num)):
#         pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q'+str(top_qubit_id)+'_Q'+str(low_qubit_id)+'_l'+str(i),pin='open'),
#                              end_pin=Dict(component='launch_zline'+str(launchpad_prefix)+str(launchpad_id-1-i),pin='tie'),),lead = Dict(start_straight=0.01,
#                              end_straight = extended_pin_length,), fillet=fillet, chip = 'main')
#         pin_side_list.append(RoutePathfinder(design,'line_launch'+str(launchpad_id-1-i)+'_pin'+str(i),options=pin_opt))

spare_launchpad_num =0
starting_launchpad_id = N-pin_qubit_num3/2-spare_launchpad_num-1
routing_enlarged_pin_launchpad(pin_qubit_num4,19,10,int(starting_launchpad_id),1,3)
routing_enlarged_pin_launchpad(pin_qubit_num5,21,70,int(starting_launchpad_id-pin_qubit_num4-1),1,4)

# routing_enlarged_pin_launchpad(pin_qubit_num6,88,79,1,int(starting_launchpad_id-pin_qubit_num4-1-pin_qubit_num5-1),5)

gui.rebuild()
gui.autoscale()

Wall time: 9min 30s


In [167]:
%%time
# design.delete_component('line_launch118'+'_readout_l3')
# design.delete_component('line_launch117'+'_pin0')
# design.delete_component('line_launch116'+'_pin1')


# side_pin_list = []
# def  get_extended_pos_y(y1,length,ydirction):  #actually it can also get x position, will correct name in future
#     y_extended = y1+length*ydirction
#     return y_extended
# fillet = '90 um'
# extended_pin_length = 0.09
# extended_pin_length_end = 0.2


# x_launch_zline28 =get_extended_pos_y(design.components['launch_zline028'].pins.tie.middle[0],extended_pin_length,
#                                               design.components['launch_zline028'].pins.tie.normal[0])
x_launch_zline18 =get_extended_pos_y(design.components['launch_zline118'].pins.tie.middle[0],extended_pin_length,
                                              design.components['launch_zline118'].pins.tie.normal[0])
side_pin_num6 = pin_qubit_num6-pin_for_side_num
left_eps = 0.15  #length left for no collision at sides

# wiring_space0 =( abs(x_launch_zline28-design.components['open_Q65_Q56_5'].parse_options().pos_x)-left_eps)/side_pin_num0
# extended_pin_length_start =wiring_space0*(side_pin_num0-1)+left_eps
total_wiring_space =abs(x_launch_zline18-design.components['open_Q88_Q79_0'].parse_options().pos_x)
wiring_space0 =( total_wiring_space-left_eps)/(pin_qubit_num6+2+2)
extended_pin_length_start =wiring_space0*(side_pin_num6-1)+left_eps

jogsS = OrderedDict()
jogsS[0] = ["L", '100um']

pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l3',pin='open'),
                         end_pin=Dict(component='launch_zline118',pin='tie'),),lead = Dict(start_straight=total_wiring_space,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch118'+'_readout_l3',options=pin_opt))


jogsS[0] = ["L", '4.6mm']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q88_Q79_0',pin='open'),
                         end_pin=Dict(component='launch_zline117',pin='tie'),),lead = Dict(start_straight=total_wiring_space-wiring_space0+0.05,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch117'+'_pin0',options=pin_opt))

jogsS[0] = ["L", '4.3mm']
# jogsS[0] = []
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q88_Q79_1',pin='open'),
                         end_pin=Dict(component='launch_zline116',pin='tie'),),lead = Dict(start_straight=total_wiring_space-2*wiring_space0,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch116'+'_pin1',options=pin_opt))

jogsS[0] = ["L", '4.0mm']
# jogsS[0] = []
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q88_Q79_2',pin='open'),
                         end_pin=Dict(component='launch_zline115',pin='tie'),),lead = Dict(start_straight=total_wiring_space-3*wiring_space0,
                         end_straight = extended_pin_length,start_jogged_extension=jogsS), fillet=fillet, chip = 'main')
side_pin_list.append(RoutePathfinder(design,'line_launch115'+'_pin2',options=pin_opt))

gui.rebuild()
gui.autoscale()

Wall time: 9min 2s


In [24]:
# wiring_space0

0.334818122871

In [168]:
%%time
#wiring side line with low virtual pins
# design.delete_component('line_pin9_low_vpin6')
side_pin_num0 = 6
side_top_pin_list=[]
left_eps = 0.15
jogsS = OrderedDict()
jogsS[0] = ["L", '3.7mm']
jogsS[1] = ["L", 2.0*wiring_space0]
jogsS[2] = ["R", '100um']
jogsE = OrderedDict()
jogsE[0] = ["L", '100um']
space_adjust = abs(design.components['open_readout_line_l7'].parse_options().pos_y-design.components['open_Q88_Q79_7'].parse_options().pos_y)
for i in range(side_pin_num0+1):
    jogsS[0] = ["L", str(3.7-0.3*i)+'mm']
    if (i==5):
        jogsS[0] = ["L", str(3.7-0.3*i-space_adjust)+'mm']
        pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l7',pin='open'),
                     end_pin=Dict(component='low_virtual'+str(i),pin='open'),),lead = Dict(start_straight=total_wiring_space-(4+i)*wiring_space0,
                     end_straight = (side_pin_num0-5+i)*extended_pin_length_end,start_jogged_extension=jogsS,),
                     fillet=fillet, chip = 'main')
        side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(3+i)+'_low_vpin'+str(i),options=pin_opt))

    elif (i==6):
        jogsS = OrderedDict()
        jogsS[0] = ["L", '100um']
        pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q90',pin='open'),
                     end_pin=Dict(component='low_virtual'+str(i),pin='open'),),lead = Dict(start_straight=left_eps+0.1,
                     end_straight = (side_pin_num0-5+i)*extended_pin_length_end,start_jogged_extension=jogsS,),
                     fillet=fillet, chip = 'main')
        side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(3+i)+'_low_vpin'+str(i),options=pin_opt))
    else:
        pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q88_Q79_'+str(3+i),pin='open'),
                         end_pin=Dict(component='low_virtual'+str(i),pin='open'),),lead = Dict(start_straight=total_wiring_space-(4+i)*wiring_space0,
                         end_straight = (side_pin_num0-5+i)*extended_pin_length_end,start_jogged_extension=jogsS,),
                         fillet=fillet, chip = 'main')
        side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(3+i)+'_low_vpin'+str(i),options=pin_opt))

#wiring readout line with low virtual pins
fillet_l = '50um'
jogsS = OrderedDict()
jogsS[0] = ["L", '100um']
pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l8',pin='open'),
             end_pin=Dict(component='low_virtual'+str(7),pin='open'),),lead = Dict(start_straight=0.05,
             end_straight = (side_pin_num0-5+7)*extended_pin_length_end,start_jogged_extension=jogsS,),
             fillet=fillet_l, chip = 'main')
side_top_pin_list.append(RoutePathfinder(design,'line_pin'+str(3+7)+'_low_vpin'+str(7),options=pin_opt))
# readout_open_space = design.components['open_readout_line_l2'].parse_options().pos_y - design.components['open_Q65_Q56_0'].parse_options().pos_y
# jogsS[0] = ["R", str(2.2-readout_open_space)+'mm']
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l2',pin='open'),
#                  end_pin=Dict(component='top_virtual6',pin='open'),),lead = Dict(start_straight=total_wiring_space-(pin_qubit_num0)*wiring_space0,
#                  end_straight = (side_pin_num0+1)*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                  fillet=fillet, chip = 'main')
# side_top_pin_list.append(RoutePathfinder(design,'readout_line_l2_top_vpin6',options=pin_opt))
gui.rebuild()
gui.autoscale()

Wall time: 9min 8s


In [170]:
side_pin_num0,top_pin_num

(6, 7)

In [171]:
%%time
#wiring between top virtual pins and launch pad 28-41
side_top_launch_list=[]
top_pin_num = 7
# wiring for launch pad 29-34
for i in range(side_pin_num0+1):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='top_virtual_l'+str(i),pin='open'),
                             end_pin=Dict(component='launch_zline0'+str(28+i),pin='tie'),),lead = Dict(start_straight=extended_pin_length,
                            end_straight =extended_pin_length,), fillet=fillet, chip = 'main')
    side_top_launch_list.append(RoutePathfinder(design,'top_vpin_'+str(i)+'launch_zline0'+str(28+i),options=pin_opt))

#wiring for launch pad 35-41
for i in range(top_pin_num):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='top_virtual_l'+str(i+7),pin='open'),
                             end_pin=Dict(component='launch_zline0'+str(35+i),pin='tie'),),lead = Dict(start_straight=extended_pin_length,
                            end_straight =extended_pin_length,), fillet=fillet, chip = 'main')
    side_top_launch_list.append(RoutePathfinder(design,'top_vpin_'+str(i+7)+'launch_zline0'+str(35+i),options=pin_opt))

gui.rebuild()
gui.autoscale()

Wall time: 10min 56s


In [172]:
%%time
#wiring between low virtual pins and launch pad 115-10
# side_top_launch_list=[]
low_pin_num = 7
# wiring for launch pad 29-34
for i in range(int(side_pin_num6+low_pin_num+3)):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='low_virtual_l'+str(i),pin='open'),
                             end_pin=Dict(component='launch_zline1'+str(14-i),pin='tie'),),lead = Dict(start_straight=extended_pin_length,
                            end_straight =extended_pin_length,), fillet=fillet, chip = 'main')
    side_top_launch_list.append(RoutePathfinder(design,'low_vpin_'+str(i)+'launch_zline1'+str(14-i),options=pin_opt))

# #wiring for launch pad 35-41
# for i in range(top_pin_num):
#     pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='top_virtual_l'+str(i+7),pin='open'),
#                              end_pin=Dict(component='launch_zline0'+str(35+i),pin='tie'),),lead = Dict(start_straight=extended_pin_length,
#                             end_straight =extended_pin_length,), fillet=fillet, chip = 'main')
#     side_top_launch_list.append(RoutePathfinder(design,'top_vpin_'+str(i+7)+'launch_zline0'+str(35+i),options=pin_opt))

gui.rebuild()
gui.autoscale()

Wall time: 12min 15s


In [66]:
# side_pin_num6

5

In [26]:
# wiring readout line 0-9
readout_line_list = []
readout_line_num = 9
for i in range(readout_line_num):
    pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_readout_line_l_rpin'+str(i),pin='open'),
                             end_pin=Dict(component='open_readout_line_r'+str(i),pin='open'),),lead = Dict(start_straight='100um',
                            end_straight ='100um',), fillet=fillet, chip = 'main')
    readout_line_list.append(RoutePathfinder(design,'readout_line_'+str(i),options=pin_opt))




# design.delete_all_components()

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_4',pin='open'),
#                          end_pin=Dict(component='top_virtual1',pin='open'),),lead = Dict(start_straight=left_eps+4*wiring_space0,
#                         end_straight = 2*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
# line_pin4_top_vpin1=RoutePathfinder(design,'line_pin4_top_vpin1',options=pin_opt)
# #
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_3',pin='open'),
#                          end_pin=Dict(component='top_virtual2',pin='open'),),lead = Dict(start_straight=left_eps+3*wiring_space0,
#                         end_straight = 3*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
# line_pin3_top_vpin2=RoutePathfinder(design,'line_pin3_top_vpin2',options=pin_opt)
# #
# #
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_2',pin='open'),
#                          end_pin=Dict(component='top_virtual3',pin='open'),),lead = Dict(start_straight=left_eps+2*wiring_space0,
#                         end_straight = 4*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
# line_pin2_top_vpin3=RoutePathfinder(design,'line_pin2_top_vpin3',options=pin_opt)
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_1',pin='open'),
#                          end_pin=Dict(component='top_virtual4',pin='open'),),lead = Dict(start_straight=left_eps+wiring_space0,
#                         end_straight = 5*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
# line_pin1_top_vpin4=RoutePathfinder(design,'line_pin1_top_vpin4',options=pin_opt)
#
# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='open_Q65_Q56_0',pin='open'),
#                          end_pin=Dict(component='top_virtual5',pin='open'),),lead = Dict(start_straight=left_eps,
#                         end_straight = 6*extended_pin_length_end,start_jogged_extension=jogsS,end_jogged_extension=jogsE),
#                         fillet=fillet, chip = 'main')
# line_pin0_top_vpin5=RoutePathfinder(design,'line_pin0_top_vpin5',options=pin_opt)
#
#
#
gui.rebuild()
gui.autoscale()

In [262]:
# for i in range(int(qubit_num/2)):
#     qubit_name.append('Q'+str(i+4*qubit_num0))
# # qubit_pos_dict_list =[]
# for i in range(int(qubit_num/2)):
#     qubit_pos_dict_list.append(dict(pos_y = design.components['Q'+str(i+1)].parse_options().pos_y-8*y1))
# qubits_copy1 = design.copy_multiple_qcomponents(qubit_list[1:int(qubit_num/2+1)],qubit_name[69:],qubit_pos_dict_list[69:])
# # qubits_copy1 = design.copy_multiple_qcomponents(qubit_list[:2],['Q23','Q24'],[dict(pos_y =5.42 ),dict(pos_y=6.75)])
# gui.rebuild()
# gui.autoscale()

In [263]:
qubit_name[69:],qubit_pos_dict_list[69:],qubit_list[:3]

(['Q92', 'Q93', 'Q94', 'Q95', 'Q96', 'Q97', 'Q98', 'Q99', 'Q100'],
 [{'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84},
  {'pos_y': -10.84}],
 [name:    Q0
  class:   New_Transomon_Cross   
  options: 
    'pos_x'             : 0,                            
    'pos_y'             : -1.355,                       
    'orientation'       : '0',                          
    'chip'              : 'main',                       
    'layer'             : '1',                          
    'connection_pads'   : {
                          },
    'cross_width'       : '1340um',                     
    'cross_height'      : '80um',                       
    'cross_inside_width': '20um',                       
    'pad_width'         : '40um',                       
    'pad_height'        : '40um',                       
    'pad_distance'      : '20um',                       